In [1]:
import pathlib
import os

import pandas as pd

import matplotlib.pyplot as plt
import datetime
%matplotlib inline

import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import mapclassify

In [2]:
import pymysql

In [3]:
db = pymysql.connect(host='localhost',
                     user='root',
                     password='password',
                     database='chicago')
cursor = db.cursor()

In [6]:
sqlcom ="SELECT * FROM weekend_morningPeak"
df= pd.read_sql(sqlcom, con=db)
print(df)

            node_id         lat         lon            timestamp value_hrf
0      001e0610ee43   41.788608  -87.598713  2019/10/05 04:00:01     66.26
1      001e061146ba    41.96759   -87.76257  2019/10/05 04:00:01     62.74
2      001e0610ba13   41.751238   -87.71299  2019/10/05 04:00:03     56.72
3      001e0610ba46   41.878377  -87.627678  2019/10/05 04:00:04     56.72
4      001e061146cb  41.9140942  -87.683022  2019/10/05 04:00:04     56.72
...             ...         ...         ...                  ...       ...
40431  001e0611462f   41.823527  -87.641054  2019/10/06 08:59:55      55.0
40432  001e061130f4   41.896157  -87.662391  2019/10/06 08:59:57     56.72
40433  001e06113ad8   41.866786  -87.666306  2019/10/06 08:59:57     56.72
40434  001e0611441e   41.808594  -87.665048  2019/10/06 08:59:58     56.72
40435  001e0611850f   41.692837  -87.613419  2019/10/06 08:59:59     62.74

[40436 rows x 5 columns]


Calculate the average sound level of each node during weekends morning peak hours

In [8]:
from decimal import Decimal
dict={}
for i in range(0,len(df)):
    key=df.iloc[i]['node_id']
    lat=df.iloc[i]['lat']
    lon=df.iloc[i]['lon']
    value=float(df.iloc[i]['value_hrf'])
    
    if key in dict:
        tempDict=dict.get(key)
        valueTemp=tempDict["value"]
        valueTemp.append(value)
        dict[key]=tempDict 
    else:
        dict[key]={"value":[value],"lon":lon,"lat":lat}

temp={}
for key in dict:
    tempDict=dict.get(key)
#     tempDict['average']=Decimal(np.mean(tempDict["value"])).quantize(Decimal('0.00'))
    tempDict['average']=np.mean(tempDict["value"])
    temp[key]=tempDict
print(temp)

{'001e0610ee43': {'value': [66.26, 62.74, 66.26, 62.74, 62.74, 66.26, 66.26, 66.26, 66.26, 56.72, 66.26, 66.26, 66.26, 66.26, 66.26, 62.74, 62.74, 66.26, 66.26, 62.74, 66.26, 56.72, 56.72, 56.72, 66.26, 66.26, 62.74, 66.26, 66.26, 56.72, 62.74, 66.26, 62.74, 62.74, 62.74, 62.74, 62.74, 56.72, 62.74, 62.74, 62.74, 66.26, 62.74, 66.26, 62.74, 66.26, 66.26, 62.74, 62.74, 62.74, 62.74, 66.26, 62.74, 62.74, 62.74, 62.74, 62.74, 66.26, 62.74, 66.26, 62.74, 66.26, 62.74, 66.26, 56.72, 62.74, 62.74, 66.26, 56.72, 66.26, 56.72, 62.74, 62.74, 66.26, 66.26, 62.74, 62.74, 62.74, 66.26, 62.74, 66.26, 62.74, 62.74, 56.72, 56.72, 62.74, 56.72, 66.26, 66.26, 66.26, 66.26, 66.26, 66.26, 62.74, 66.26, 62.74, 62.74, 62.74, 56.72, 66.26, 62.74, 56.72, 62.74, 62.74, 62.74, 56.72, 62.74, 62.74, 56.72, 62.74, 62.74, 66.26, 62.74, 66.26, 62.74, 62.74, 62.74, 66.26, 66.26, 66.26, 62.74, 62.74, 62.74, 56.72, 66.26, 62.74, 56.72, 62.74, 62.74, 56.72, 66.26, 62.74, 62.74, 66.26, 66.26, 66.26, 62.74, 62.74, 56.72,

In [9]:
df=pd.DataFrame.from_dict(temp,orient='index')
df

,value,lon,lat,average
001e0610ee43,"[66.26, 62.74, 66.26, 62.74, 62.74, 66.26, 66....",-87.598713,41.788608,61.802999
001e061146ba,"[62.74, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.76257,41.96759,56.361616
001e0610ba13,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.71299,41.751238,56.770695
001e0610ba46,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.627678,41.878377,60.140559
001e061146cb,"[56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 56.7...",-87.683022,41.9140942,60.730618
001e06117b44,"[62.74, 62.74, 62.74, 66.26, 62.74, 62.74, 62....",-87.66263000000002,41.721301,60.695556
001e061183f5,"[62.74, 62.74, 62.74, 62.74, 62.74, 62.74, 62....",-87.62102,41.692703,60.826568
001e06113a24,"[56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 56.7...",-87.597995,41.788979,56.924265
001e061144cd,"[55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 56.72, 56...",-87.666599,41.8742,56.683300
001e06113ad8,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.666306,41.866786,56.756530


In [10]:
df.shape

(31, 4)

In [11]:
df['average']

001e0610ee43    61.802999
001e061146ba    56.361616
001e0610ba13    56.770695
001e0610ba46    60.140559
001e061146cb    60.730618
001e06117b44    60.695556
001e061183f5    60.826568
001e06113a24    56.924265
001e061144cd    56.683300
001e06113ad8    56.756530
001e0610bbe5    56.756963
001e0610e538    58.988696
001e06113ace    56.877028
001e0611441e    58.287556
001e0610ba15    56.655112
001e0610f732    63.273273
001e061144be    56.730260
001e06118295    59.163047
001e0610f05c    61.220199
001e06113107    56.488367
001e061130f4    57.918120
001e0610ee36    57.448073
001e0610bc10    87.298643
001e06113d20    56.834911
001e0611462f    56.404625
001e06113acb    58.034429
001e0610f6db    56.999485
001e0610f703    61.106696
001e06112e77    56.856202
001e061184a3    62.258400
001e0611850f    61.060229
Name: average, dtype: float64